<a href="https://colab.research.google.com/github/tjose185/Kawa_Space/blob/main/KawaSpace_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To calculate the MSAVI2 index I am using Landsat 8 data for the given locations. To download Landsat8 data I am using Google Earth Engine. geemap is a python package for interactive mapping with google earth engine. geemap package is used to query and download Landsat data.
ie, to select the images for the given location for a given time period, for the given area of interest, remove cloudy images etc. 
Then we can download the images to either our system or to the google drive.
 I am using Google Colab platform to run the code.Installing the geemap package using pip

In [ ]:
pip install geemap                      

Import necesary packages 

In [ ]:
import ee                               
import geemap
import os

Connect with the google earth engine. You need to have google earth engine account created.

In [ ]:
Map = geemap.Map()                      
Map

The location data is stored into variables. we can change the roi to generate MSAVI2 index for differnet locations. Here I have taken location 2 as roi.

In [ ]:
loc1 = ee.Geometry.Polygon([ [75.45157492160797,30.634404129904425],[75.4524278640747,30.63372099804389],
                             [75.45236885547638,30.634399514164798],[75.45157492160797,30.634404129904425] ])
loc2 = ee.Geometry.Polygon([ [85.85590660572052,26.093581136401006],[85.85663080215454,26.09337879451938 ],
                             [85.85686147212981,26.094009907326967],[85.85622847080231,26.09425078918021]  ])
loc3 = ee.Geometry.Polygon([ [78.6662346124649,17.6686911655875],[78.6662346124649,17.66929686130703],
                             [78.66571158170699,17.66929686130703],[78.66571158170699,17.66869116558751]  ])

Here we are collecting all the images of landsat8 and then filter the images based on area of interest and date. We collect images of landsat 8 captured only during the given period. Also we can filter the images based on cloud cover if needed.

In [ ]:
roi = loc2
collection = ee.ImageCollection ("LANDSAT/LC08/C01/T1") \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-06-30') #\
    #.filter(ee.Filter.metadata_("CLOUD_COVER", "less_than", 10))

 Here we retrieve the ID of the images. We can see how many images are there.

In [ ]:
print(collection.aggregate_array('system:index').getInfo())          

Since the roi is covered by two diiferent tiles, we select images from one tile only.

In [ ]:
list = collection.aggregate_array('system:index').getInfo()          
image_scene = list[0:9]
print(image_scene)

The selected images can be downloaded to either our computer or to the drive. An roi can be given to download only the necessary data.

In [ ]:
for scene in image_scene:                                                                             
  data1 = ee.Image('LANDSAT/LC08/C01/T1/'+scene)                                                      
  roi = loc2
  geemap.ee_export_image_to_drive(data1, description='%s' %scene, folder='LandsatData', region=roi, scale=10)

Pyrsgis is a python package used to read, write and for doing some basic functions on raster data. Installing the package using pip.

In [ ]:
pip install pyrsgis                          

Here we are connecting the google drive with the google colab so that we can access the data in google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/LandsatData/

 The packages are imported. To calculate the MSAVI2 index we need to take the NIR and Red Bands from each rasters. Here I have taken NIR and Red bands from an image and read it as a numpy array. The raster information such as the projection are stored in the variable 'ds'. Next we can calculate the MSAVI using the equation. Last we can export the calculated MSAVI as raster to the drive.

In [ ]:
from pyrsgis import raster                                                     
import numpy as np                                                              
import math                                                                     
for scene in image_scene:                                                  
  file = scene+'.tif'
  print(file)
  print(RedBand)
  print(NIRBand)
  ds, RedBand = raster.read(file, bands=4)
  ds, NIRBand = raster.read(file, bands=5)
  MSAVI = 1/2*(2*NIRBand+1-np.sqrt(np.square((2*NIRBand+1), dtype='int32')-8*(NIRBand-RedBand)))
  raster.export(MSAVI, ds, '/content/drive/My Drive/LandsatData/MSAVI_'+scene+'.tif', dtype='float64', bands='all') 


I could only do this much. I was not able to create a time series graph of each location.